PLEASE COMMENT OUT THE CELLS IN WHICH I AM CALLING THE 100K DATASET IF YOU WISH TO SEE HOW THEY PERFORM!
BUT I COMMENTED HOW MUCH TIME THEY TOOK!

In [1]:
import numpy as np
import pandas as pd

In [2]:
#importing the movieratings dataset with index col=0 so I can better work with it
movieratings=pd.read_csv('movieratings.csv', sep=",")

In [3]:
# please comment out to read the 100K dataset
# # reading the 2 datasets by setting the column names
# rcolumns = ['User', 'movie_id', 'rating', 'unix_timestamp']
# ratings = pd.read_csv('u.data', sep='\t', names=rcolumns)
# mcolumns = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
# movies = pd.read_csv('u.item', sep='|', names=mcolumns, usecols=range(5),encoding='latin-1')
# #merging movies and ratings datasets
# movie_ratings = pd.merge(movies, ratings)
# #only taking into account the columns that we need for our recom
# movie_ratings=movie_ratings.loc[:,['title','User','rating']]
# #transforming the shape of our dataframe so we can have the same shape as the "movieratings" one!
# hundredKmovie=movie_ratings.reset_index().groupby(['User', 'title'])['rating'].aggregate('first').unstack()
# #as opposed to smaller dataset we don't have names here, hence we transform the index to strings to have names of people rather than numbers even if it's only ids
# hundredKmovie.index=hundredKmovie.index.map(str)

In [4]:
# function that returns the similarity between 2 users, the similarity is computed from -1 to 1
# the function will return a zero in case of no Similarity between the 2 users!
def pearsonSimilarity(prefs,person1,person2):
    #just in case the 'User' is not in index, I added it!
    if 'User' in prefs.columns:
        prefs['User']=prefs['User'].astype(str)
        prefs=prefs.set_index('User')
    #we only take into account the ratings of these 2 persons and we also drop all NaN
    #from both of their ratings so we can only have movies they both watched    
    df_1=prefs.loc[[person1, person2],:].dropna(axis='columns')
    #we are measuring the correlation between the remaining ratings of person 1 and person 2
    scores=df_1.loc[person1].corr(df_1.loc[person2])
    #if we don't have any similar rating or if the standard deviation is 0, then we return a 0 instead of Nan
    if np.isnan(scores)==True:
            scores=0
    return scores

In [5]:
# #calling our function to test it with the 100K dataset
# person_1='8'
# person_2='4'
# pearsonSimilarity(hundredKmovie,person_1,person_2)
# # #result: 0.1178511301977579

In [6]:
#calling our function to test it with the 20x20 dataset
person1='4656: Victoria'
person2='139: Nuria'
pearsonSimilarity(movieratings,person1,person2)
#result: 0.36514837167011077

0.36514837167011077

In [7]:
# function that returns recommendations for the movies that 'person' didn't watch!
# in the case that no one watched the movie yet, it returns a 0 value for that movie!
def getRecommendations(prefs,person,similarity=pearsonSimilarity):
    #just in case the 'User' is not in index, I added it!
    if 'User' in prefs.columns:
        prefs['User']=prefs['User'].astype(str)
        prefs=prefs.set_index('User')
    #creating a dataframe that has all the movies that 'person' watched! Starting by taking into account only
    #his ratings then by dropping NaN(movies he didn't watch) 
    titlesperson=pd.DataFrame(prefs.loc[person,:].dropna())
    #finally droping the person column from titlesperson! so we get a 1 column dataset!
    titlesperson=titlesperson.drop(columns=[person])
    replica=prefs
    #iterating over all movies that the person watched and dropping them from the dataframe
    for title in titlesperson.index:
            replica=replica.drop(columns=title)
    #creating 2 lists to use in the for loop
    list_1=[]
    list_2=[]
    #iterating over every user, in order to compute the similarity of person with each of the other users!
    for i in replica.index:
        #checking that we're not iterating over the same person that we're getting recomm for
        if i != person:
            #we then call the similarity function in order to get the similarities
            list_1=(i,similarity(prefs,person,i))
            list_2.append(list_1)
            
    #we create a dataframe out of recommendations and merge it with the initial dataset
    list_2=pd.DataFrame(list_2)
    list_2.columns=['User','Similarity']
    list_2=list_2.dropna()
    #replacing the negative similarities with 0! because we only want to make positive recom
    list_2.loc[list_2['Similarity'] <0] = 0
    #merging the 2 datasets
    merged = pd.DataFrame.merge( list_2,replica, on='User',left_on=None, right_on=None,)
    #filling the Nan with 0 because we will multiply later and they would be not taken into consideration
    merged=merged.fillna(0)
    list_3=[]
    list_4=[]
    #iterating over all the columns, in order to apply the formula to each of the movies!
    #and get the recommendations!
    for j in merged.columns:
        #making sure that we don't iterate over the 'User' and the Similarity one!
        if (j != 'User') and (j!='Similarity'):
            #creating a new column that is equal to 1 if the rating is different than 0 and 0 otherwise
            merged.loc[merged[j] !=0, 'SIM'] = 1
            merged.loc[merged[j] ==0, 'SIM'] = 0
            #checking that our denominator is not 0! if it is we return the value 0 for that movie!
            if sum(merged['Similarity'].mul(merged['SIM'])) <= 0.01:
                r=0
            else:
                #only taking into account the similarity of the users who rated that specific movie!
                value_1=merged['Similarity'].mul(merged[j])
                r=(sum(value_1)/sum(merged['Similarity'].mul(merged['SIM'])))
                # set the upper threshold to 5 because in some cases it reached 5.0000001
                if r > 5:
                    r=5
            list_3=(r,j)
            list_4.append(list_3)
    rankings = sorted(list_4, key=lambda tup: tup[0],reverse=True)

    return rankings

In [8]:
#calling our function to test it with the 20x20 dataset
# it takes 0.16 seconds to run
# import time
# start_time = time.time()
person1='4656: Victoria'
getRecommendations(movieratings,person1,similarity=pearsonSimilarity)
# print ("My program took", time.time() - start_time, "to run")

[(3.804929227387678, '2571: Matrix, The (1999)'),
 (3.252404115805032, '318: Shawshank Redemption, The (1994)'),
 (3.001761291308658, '356: Forrest Gump (1994)'),
 (2.9987765717943913, '2396: Shakespeare in Love (1998)'),
 (2.775452598428217, '2762: Sixth Sense, The (1999)'),
 (2.1518724871008907, '541: Blade Runner (1982)'),
 (1.796481125969382, '2028: Saving Private Ryan (1998)')]

In [9]:
# RESULT JUST IN CASE YOU WERE NOT ABLE TO LOAD THEM
# [(3.804929227387678, '2571: Matrix, The (1999)'),
#  (3.252404115805032, '318: Shawshank Redemption, The (1994)'),
#  (3.001761291308658, '356: Forrest Gump (1994)'),
#  (2.9987765717943913, '2396: Shakespeare in Love (1998)'),
#  (2.775452598428217, '2762: Sixth Sense, The (1999)'),
#  (2.1518724871008907, '541: Blade Runner (1982)'),
#  (1.796481125969382, '2028: Saving Private Ryan (1998)')]

In [10]:
# # calling our function to test it with the 100K dataset
# #takes 17.37 seconds to run!
# # import time
# # start_time = time.time()
# person_1='8'
# person_2='4'
# getRecommendations(hundredKmovie,person_2,similarity=pearsonSimilarity)
# # print ("My program took", time.time() - start_time, "to run")

In [11]:
# function that returns the n Users that have the highest similarity with 'person'
# the similarities are from -1 to 1!
# if no User have similarities with 'person', it will return zeroes
def topMatches(prefs,person,n=5,similarity=pearsonSimilarity): 
    list_1=[]
    list_2=[]
    #just in case the 'User' is not in index, I added it!
    if 'User' in prefs.columns:
        prefs['User']=prefs['User'].astype(str)
        prefs=prefs.set_index('User')
    #iterating over all the users except the person we're checking the matches for!
    for i in prefs.index:
        if i != person:
            #getting the similarity of 'person' with all the other users
            list_1=(similarity(prefs,person,i),i)
            #if the result is NaN (for some reason) we don't append in the list adn we move to the next user!
            if np.isnan(list_1[0])==False:
                list_2.append(list_1)
    scores = sorted(list_2, key=lambda tup: tup[0],reverse=True)
    return scores[0:n]

In [12]:
#calling our function to test it with the 20x20 dataset
person1='4656: Victoria'
n=5
topMatches(movieratings,person1,n=5,similarity=pearsonSimilarity)

[(0.944911182523068, '6037: Rachel'),
 (0.5976143046671967, '3823: Ana'),
 (0.578979445733232, '4790: Oriol'),
 (0.492592183071889, '5277: Maria'),
 (0.4273246726830629, '3118: Carles')]

In [13]:
# RESULTS JUST IN CASE YOU WERE NOT ABLE TO LOAD THEM
# [(0.944911182523068, '6037: Rachel'),
#  (0.5976143046671967, '3823: Ana'),
#  (0.578979445733232, '4790: Oriol'),
#  (0.492592183071889, '5277: Maria'),
#  (0.4273246726830629, '3118: Carles')]

In [14]:
# #calling our function to test it with the 100K dataset
# person_1='8'
# person_2='4'
# topMatches(hundredKmovie,person_1,n=5,similarity=pearsonSimilarity)